##  Binary Classification using Perception, MLP and Autoencoder

#### Loading the required libraries

In [1]:
import os

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.models import Sequential, Model
from keras.layers import Dense, Input

#### Problem

    Is to identify products at risk of backorder before the event occurs so the business has time to react. 

#### Data

Data file contains the historical data for the 8 weeks prior to the week we are trying to predict. The data was taken as weekly snapshots at the start of each week. Columns are defined as follows:

    sku - Random ID for the product

    national_inv - Current inventory level for the part

    lead_time - Transit time for product (if available)

    in_transit_qty - Amount of product in transit from source

    forecast_3_month - Forecast sales for the next 3 months

    forecast_6_month - Forecast sales for the next 6 months

    forecast_9_month - Forecast sales for the next 9 months

    sales_1_month - Sales quantity for the prior 1 month time period

    sales_3_month - Sales quantity for the prior 3 month time period

    sales_6_month - Sales quantity for the prior 6 month time period

    sales_9_month - Sales quantity for the prior 9 month time period

    min_bank - Minimum recommend amount to stock

    potential_issue - Source issue for part identified

    pieces_past_due - Parts overdue from source

    perf_6_month_avg - Source performance for prior 6 month period

    perf_12_month_avg - Source performance for prior 12 month period

    local_bo_qty - Amount of stock orders overdue

    deck_risk - Part risk flag

    oe_constraint - Part risk flag

    ppap_risk - Part risk flag

    stop_auto_buy - Part risk flag

    rev_stop - Part risk flag

    went_on_backorder - Product actually went on backorder. This is the target value.

#### Identify Right Error Metrics

    Based on the businees have to identify right error metrics.

#### Loading the data

In [2]:
data = pd.read_csv("/home/datasets/lab/DT_BackOrders.csv",header=0)

#### Understand the Data

See the No. row and columns

In [3]:
data.shape

(61589, 23)

Display the columns

In [4]:
data.columns

Index(['sku', 'national_inv', 'lead_time', 'in_transit_qty',
       'forecast_3_month', 'forecast_6_month', 'forecast_9_month',
       'sales_1_month', 'sales_3_month', 'sales_6_month', 'sales_9_month',
       'min_bank', 'potential_issue', 'pieces_past_due', 'perf_6_month_avg',
       'perf_12_month_avg', 'local_bo_qty', 'deck_risk', 'oe_constraint',
       'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder'],
      dtype='object')

Display the index

In [5]:
data.index

RangeIndex(start=0, stop=61589, step=1)

See the top rows of the data

In [6]:
data.head(3)

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
0,1888279,117,NaN,0,0,0,0,0,0,15,...,0,-99.00,-99.00,0,No,No,Yes,Yes,No,No
1,1870557,7,2.0,0,0,0,0,0,0,0,...,0,0.50,0.28,0,Yes,No,No,Yes,No,No
2,1475481,258,15.0,10,10,77,184,46,132,256,...,0,0.54,0.70,0,No,No,No,Yes,No,No


Shows a quick statistic summary of your data using describe

In [7]:
data.describe(include='all')

,sku,national_inv,lead_time,in_transit_qty,forecast_3_month,forecast_6_month,forecast_9_month,sales_1_month,sales_3_month,sales_6_month,...,pieces_past_due,perf_6_month_avg,perf_12_month_avg,local_bo_qty,deck_risk,oe_constraint,ppap_risk,stop_auto_buy,rev_stop,went_on_backorder
count,6.158900e+04,61589.000000,58186.000000,61589.000000,6.158900e+04,6.158900e+04,6.158900e+04,61589.000000,61589.000000,6.158900e+04,...,61589.000000,61589.000000,61589.000000,61589.000000,61589,61589,61589,61589,61589,61589
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2,2,2,2,2,2
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,No,No,No,Yes,No,No
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,48145,61577,53792,59303,61569,50296
mean,2.037188e+06,287.721882,7.559619,30.192843,1.692728e+02,3.150413e+02,4.535760e+02,44.742957,150.732631,2.835465e+02,...,1.605400,-6.264182,-5.863664,1.205361,NaN,NaN,NaN,NaN,NaN,NaN
std,6.564178e+05,4233.906931,6.498952,792.869253,5.286742e+03,9.774362e+03,1.420201e+04,1373.805831,5224.959649,8.872270e+03,...,42.309229,25.537906,24.844514,29.981155,NaN,NaN,NaN,NaN,NaN,NaN
min,1.068628e+06,-2999.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000,-99.000000,-99.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.498574e+06,3.000000,4.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000,0.620000,0.640000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.898033e+06,10.000000,8.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,2.000000,4.000000e+00,...,0.000000,0.820000,0.800000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,2.314826e+06,57.000000,8.000000,0.000000,1.200000e+01,2.500000e+01,3.600000e+01,6.000000,17.000000,3.400000e+01,...,0.000000,0.960000,0.950000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN


Display data type of each variable

In [8]:
data.dtypes

sku                    int64
national_inv           int64
lead_time            float64
in_transit_qty         int64
forecast_3_month       int64
forecast_6_month       int64
forecast_9_month       int64
sales_1_month          int64
sales_3_month          int64
sales_6_month          int64
sales_9_month          int64
min_bank               int64
potential_issue       object
pieces_past_due        int64
perf_6_month_avg     float64
perf_12_month_avg    float64
local_bo_qty           int64
deck_risk             object
oe_constraint         object
ppap_risk             object
stop_auto_buy         object
rev_stop              object
went_on_backorder     object
dtype: object

#### Observations

    sku is Categorical but is interpreted as int64 
    potential_issue, deck_risk, oe_constraint, ppap_risk, stop_auto_buy, rev_stop, and went_on_backorder are also categorical but is interpreted as object. 

#### Convert all the attributes to appropriate type

Data type conversion

    Using astype('category') to convert potential_issue, deck_risk, oe_constraint, ppap_risk, stop_auto_buy, rev_stop, and went_on_backorder attributes to categorical attributes.


In [9]:
for col in ['sku', 'potential_issue', 'deck_risk', 'oe_constraint', 'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder']:
    data[col] = data[col].astype('category')

Display data type of each variable

In [10]:
data.dtypes

sku                  category
national_inv            int64
lead_time             float64
in_transit_qty          int64
forecast_3_month        int64
forecast_6_month        int64
forecast_9_month        int64
sales_1_month           int64
sales_3_month           int64
sales_6_month           int64
sales_9_month           int64
min_bank                int64
potential_issue      category
pieces_past_due         int64
perf_6_month_avg      float64
perf_12_month_avg     float64
local_bo_qty            int64
deck_risk            category
oe_constraint        category
ppap_risk            category
stop_auto_buy        category
rev_stop             category
went_on_backorder    category
dtype: object

#### Delete sku attribute

In [11]:
np.size(np.unique(data.sku, return_counts=True)[0])

61589

In [12]:
data.drop('sku', axis=1, inplace=True)

#### Missing Data

    Missing value analysis and dropping the records with missing values

In [13]:
data.isnull().sum()

national_inv            0
lead_time            3403
in_transit_qty          0
forecast_3_month        0
forecast_6_month        0
forecast_9_month        0
sales_1_month           0
sales_3_month           0
sales_6_month           0
sales_9_month           0
min_bank                0
potential_issue         0
pieces_past_due         0
perf_6_month_avg        0
perf_12_month_avg       0
local_bo_qty            0
deck_risk               0
oe_constraint           0
ppap_risk               0
stop_auto_buy           0
rev_stop                0
went_on_backorder       0
dtype: int64

Observing the number of records before and after missing value records removal

In [14]:
print (data.shape)

(61589, 22)


Since the number of missing values is about 5%. For initial analysis we ignore all these records

In [15]:
data = data.dropna(axis=0)

In [16]:
data.isnull().sum()
print(data.shape)

(58186, 22)


#### Converting Categorical to Numeric

For some of the models all the independent attribute should be of type numeric and Linear Regression model is one among them.
But this data set has some categorial attributes.

'pandas.get_dummies' To convert convert categorical variable into dummy/indicator variables


In [17]:
print (data.columns)

Index(['national_inv', 'lead_time', 'in_transit_qty', 'forecast_3_month',
       'forecast_6_month', 'forecast_9_month', 'sales_1_month',
       'sales_3_month', 'sales_6_month', 'sales_9_month', 'min_bank',
       'potential_issue', 'pieces_past_due', 'perf_6_month_avg',
       'perf_12_month_avg', 'local_bo_qty', 'deck_risk', 'oe_constraint',
       'ppap_risk', 'stop_auto_buy', 'rev_stop', 'went_on_backorder'],
      dtype='object')


Creating dummy variables.

    If we have k levels in a category, then we create k-1 dummy variables as the last one would be redundant. So we use the parameter drop_first in pd.get_dummies function that drops the first level in each of the category


In [18]:
categorical_Attributes = data.select_dtypes(include=['category']).columns

In [19]:
data = pd.get_dummies(columns=categorical_Attributes, data=data, prefix=categorical_Attributes, prefix_sep="_",drop_first=True)

In [20]:
print (data.columns, data.shape)

Index(['national_inv', 'lead_time', 'in_transit_qty', 'forecast_3_month',
       'forecast_6_month', 'forecast_9_month', 'sales_1_month',
       'sales_3_month', 'sales_6_month', 'sales_9_month', 'min_bank',
       'pieces_past_due', 'perf_6_month_avg', 'perf_12_month_avg',
       'local_bo_qty', 'potential_issue_Yes', 'deck_risk_Yes',
       'oe_constraint_Yes', 'ppap_risk_Yes', 'stop_auto_buy_Yes',
       'rev_stop_Yes', 'went_on_backorder_Yes'],
      dtype='object') (58186, 22)


#### Target attribute distribution

In [21]:
pd.value_counts(data['went_on_backorder_Yes'].values)

0    47217
1    10969
dtype: int64

#### Split the data in to train and test

sklearn.model_selection.train_test_split

    Split arrays or matrices into random train and test subsets

In [22]:
#Performing train test split on the data
X, y = data.loc[:,data.columns!='went_on_backorder_Yes'].values, data.loc[:,'went_on_backorder_Yes'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)


In [23]:
#To get the distribution in the target in train and test
print(pd.value_counts(y_train))
print(pd.value_counts(y_test))

0    33067
1     7663
dtype: int64
0    14150
1     3306
dtype: int64


In [24]:
X_train.shape[1]

21

#### Perceptron Model Building

In [25]:
perceptron_model = Sequential()

perceptron_model.add(Dense(1, input_dim=X_train.shape[1], activation='sigmoid', kernel_initializer='normal'))

In [26]:
perceptron_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 22        
Total params: 22
Trainable params: 22
Non-trainable params: 0
_________________________________________________________________


In [27]:
perceptron_model.compile(loss='binary_crossentropy', optimizer='adam')

In [28]:
perceptron_model.fit(X_train, y_train, epochs=100)

Epoch 1/100
1273/1273 [==============================] - 2s 925us/step - loss: 1.2334
Epoch 2/100
1273/1273 [==============================] - 1s 990us/step - loss: 1.9043 0s - loss - ETA: 0s - loss: 1
Epoch 3/100
1273/1273 [==============================] - 1s 739us/step - loss: 1.1566
Epoch 4/100
1273/1273 [==============================] - 1s 781us/step - loss: 0.8733 0s - los
Epoch 5/100
1273/1273 [==============================] - 1s 821us/step - loss: 0.9279
Epoch 6/100
1273/1273 [==============================] - 1s 828us/step - loss: 1.0194
Epoch 7/100
1273/1273 [==============================] - 2s 1ms/step - loss: 1.9256
Epoch 8/100
1273/1273 [==============================] - 1s 1ms/step - loss: 2.2561
Epoch 9/100
1273/1273 [==============================] - 2s 1ms/step - loss: 1.0457
Epoch 10/100
1273/1273 [==============================] - 1s 1ms/step - loss: 0.8028
Epoch 11/100
1273/1273 [==============================] - 1s 991us/step - loss: 1.7582
Epoch 12/100
1273/127

1273/1273 [==============================] - 2s 1ms/step - loss: 0.7524
Epoch 96/100
1273/1273 [==============================] - 2s 1ms/step - loss: 1.0437
Epoch 97/100
1273/1273 [==============================] - 2s 1ms/step - loss: 1.1947
Epoch 98/100
1273/1273 [==============================] - 1s 1ms/step - loss: 1.0221
Epoch 99/100
1273/1273 [==============================] - 1s 1ms/step - loss: 1.5547
Epoch 100/100
1273/1273 [==============================] - 1s 1ms/step - loss: 1.6427


#### Predictions

In [29]:
test_pred=perceptron_model.predict_classes(X_test)
train_pred=perceptron_model.predict_classes(X_train)

/usr/share/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


#### Getting evaluation metrics and evaluating model performance

In [30]:
confusion_matrix_test = confusion_matrix(y_test, test_pred)
confusion_matrix_train = confusion_matrix(y_train, train_pred)

print(confusion_matrix_train)
print(confusion_matrix_test)

[[32702   365]
 [ 6468  1195]]
[[13987   163]
 [ 2801   505]]


#### Calculate Accuracy, True Positive Rate and True Negative Rates

In [31]:
Accuracy_Train = (confusion_matrix_train[0,0]+confusion_matrix_train[1,1])/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1]+confusion_matrix_train[1,0]+confusion_matrix_train[1,1])
TNR_Train = confusion_matrix_train[0,0]/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1])
TPR_Train = confusion_matrix_train[1,1]/(confusion_matrix_train[1,0]+confusion_matrix_train[1,1])

print("Train TNR: ",TNR_Train)
print("Train TPR: ",TPR_Train)
print("Train Accuracy: ",Accuracy_Train)

Train TNR:  0.9889618048205159
Train TPR:  0.1559441472008352
Train Accuracy:  0.8322366805794255


In [32]:
Accuracy_Test = (confusion_matrix_test[0,0]+confusion_matrix_test[1,1])/(confusion_matrix_test[0,0]+confusion_matrix_test[0,1]+confusion_matrix_test[1,0]+confusion_matrix_test[1,1])
TNR_Test = confusion_matrix_test[0,0]/(confusion_matrix_test[0,0] +confusion_matrix_test[0,1])
TPR_Test = confusion_matrix_test[1,1]/(confusion_matrix_test[1,0] +confusion_matrix_test[1,1])

print("Test TNR: ",TNR_Test)
print("Test TPR: ",TPR_Test)
print("Test Accuracy: ",Accuracy_Test)

Test TNR:  0.9884805653710247
Test TPR:  0.15275257108287962
Test Accuracy:  0.8302016498625114


#### Derive new non-linear features using autoencoder

In [33]:
# The size of encoded and actual representations
encoding_dim = 16 
actual_dim = X_train.shape[1]

In [34]:
# Input placeholder
input_attrs = Input(shape=(actual_dim,))

# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_attrs)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(actual_dim, activation='sigmoid')(encoded)

In [35]:
# this model maps an input to its reconstruction
autoencoder = Model(input_attrs, decoded)

In [36]:
print(autoencoder.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 21)]              0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                352       
_________________________________________________________________
dense_2 (Dense)              (None, 21)                357       
Total params: 709
Trainable params: 709
Non-trainable params: 0
_________________________________________________________________
None


In [37]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [38]:
autoencoder.fit(X_train, X_train, epochs=100)

Epoch 1/100
1273/1273 [==============================] - 2s 1ms/step - loss: -26577717.6501A: 0s - loss: -27507
Epoch 2/100
1273/1273 [==============================] - 2s 1ms/step - loss: -27629986.7584
Epoch 3/100
1273/1273 [==============================] - 2s 1ms/step - loss: -201953637.9185
Epoch 4/100
1273/1273 [==============================] - 2s 2ms/step - loss: -30035467.6425
Epoch 5/100
1273/1273 [==============================] - 2s 1ms/step - loss: -344395339.7863
Epoch 6/100
1273/1273 [==============================] - 2s 2ms/step - loss: -342929999.6622
Epoch 7/100
1273/1273 [==============================] - 2s 1ms/step - loss: -228064267.0427
Epoch 8/100
1273/1273 [==============================] - 2s 1ms/step - loss: -781294300.8020
Epoch 9/100
1273/1273 [==============================] - 1s 991us/step - loss: -596244520.6020
Epoch 10/100
1273/1273 [==============================] - 1s 1ms/step - loss: -103951829.2950
Epoch 11/100
1273/1273 [==========================

1273/1273 [==============================] - 2s 1ms/step - loss: -8454289936.2386
Epoch 86/100
1273/1273 [==============================] - 2s 1ms/step - loss: -4353215165.6334
Epoch 87/100
1273/1273 [==============================] - 2s 1ms/step - loss: -26755591730.1350
Epoch 88/100
1273/1273 [==============================] - 2s 1ms/step - loss: -48073018652.2323
Epoch 89/100
1273/1273 [==============================] - 2s 2ms/step - loss: -29829917817.5950
Epoch 90/100
1273/1273 [==============================] - 2s 1ms/step - loss: -23617421548.2072
Epoch 91/100
1273/1273 [==============================] - 2s 1ms/step - loss: -10121782453.3626
Epoch 92/100
1273/1273 [==============================] - 2s 1ms/step - loss: -20651801369.2810
Epoch 93/100
1273/1273 [==============================] - 2s 1ms/step - loss: -10190460346.5746
Epoch 94/100
1273/1273 [==============================] - 2s 1ms/step - loss: -10056434908.1570
Epoch 95/100
1273/1273 [==============================]

Create a separate encoder model

In [39]:
# this model maps an input to its encoded representation
encoder = Model(input_attrs, encoded)

In [40]:
print(encoder.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 21)]              0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                352       
Total params: 352
Trainable params: 352
Non-trainable params: 0
_________________________________________________________________
None


#### derive new non-linear features

In [41]:
X_train_nonLinear_features = encoder.predict(X_train)
X_test_nonLinear_features = encoder.predict(X_test)

In [42]:
X_train_nonLinear_features[1:2,:]

array([[1132.7853, 1210.6514, 1257.5206, 1198.3643, 1139.1997, 1282.446 ,
        1219.8794, 1180.4851, 1261.1199, 1225.3845, 1129.0045, 1213.9185,
        1248.2655, 1256.0734, 1263.2224, 1233.0189]], dtype=float32)

In [43]:
encoder.get_weights()

[array([[  3.0438576,   3.6714492,   3.4604812,   3.284401 ,   3.0981462,
           3.4374838,   3.6635752,   3.053891 ,   3.0580802,   3.5715654,
           2.9090643,   3.0019627,   3.6111887,   3.498414 ,   3.7332194,
           3.1128902],
        [ 20.892052 ,  23.88674  ,  23.930578 ,  22.084517 ,  22.937416 ,
          27.233696 ,  24.8992   ,  24.368177 ,  23.799368 ,  23.610823 ,
          21.974562 ,  22.886496 ,  24.70836  ,  22.135155 ,  23.65166  ,
          23.149393 ],
        [  2.8695116,   2.7085586,   2.689878 ,   3.3630993,   2.3891144,
           3.5577621,   3.4534807,   3.1380975,   3.4128444,   3.110708 ,
           2.9169302,   3.0767474,   3.0892327,   2.970638 ,   3.1258504,
           2.8528163],
        [  2.7379303,   3.1949327,   3.547045 ,   3.1567545,   3.1136446,
           2.8804812,   3.39318  ,   3.332313 ,   3.2128358,   3.0116868,
           3.0083053,   3.3608088,   3.2904294,   3.1607378,   3.046194 ,
           2.8190832],
        [  2.8375728

#### Combining new non-linear features to X_train and X_test respectively

In [44]:
X_train = np.concatenate((X_train, X_train_nonLinear_features), axis=1)
X_test = np.concatenate((X_test, X_test_nonLinear_features), axis=1)

#### Perceptron Model Building with both actual and non-linear features

In [45]:
perceptron_model = Sequential()

perceptron_model.add(Dense(1, input_dim=X_train.shape[1], activation='sigmoid'))

In [46]:
perceptron_model.compile(loss='binary_crossentropy', optimizer='adam')

In [47]:
perceptron_model.fit(X_train, y_train, epochs=100)

Epoch 1/100
1273/1273 [==============================] - 2s 1ms/step - loss: 2420.1597
Epoch 2/100
1273/1273 [==============================] - 1s 1ms/step - loss: 39.5215
Epoch 3/100
1273/1273 [==============================] - 1s 954us/step - loss: 74.7435
Epoch 4/100
1273/1273 [==============================] - 1s 1ms/step - loss: 40.1136
Epoch 5/100
1273/1273 [==============================] - 1s 1ms/step - loss: 46.7896
Epoch 6/100
1273/1273 [==============================] - 2s 1ms/step - loss: 23.5892
Epoch 7/100
1273/1273 [==============================] - 2s 1ms/step - loss: 33.7747
Epoch 8/100
1273/1273 [==============================] - 2s 1ms/step - loss: 41.1347
Epoch 9/100
1273/1273 [==============================] - 2s 1ms/step - loss: 11.6426
Epoch 10/100
1273/1273 [==============================] - 2s 1ms/step - loss: 27.0195
Epoch 11/100
1273/1273 [==============================] - 2s 1ms/step - loss: 96.2471
Epoch 12/100
1273/1273 [==============================] - 2

1273/1273 [==============================] - 1s 1ms/step - loss: 13.6043
Epoch 96/100
1273/1273 [==============================] - 1s 1ms/step - loss: 22.1022
Epoch 97/100
1273/1273 [==============================] - 1s 1ms/step - loss: 38.3838
Epoch 98/100
1273/1273 [==============================] - 1s 1ms/step - loss: 24.7133
Epoch 99/100
1273/1273 [==============================] - 2s 1ms/step - loss: 15.5535
Epoch 100/100
1273/1273 [==============================] - 2s 1ms/step - loss: 24.9712


#### Predictions

In [48]:
test_pred=perceptron_model.predict_classes(X_test)
train_pred=perceptron_model.predict_classes(X_train)

/usr/share/anaconda3/lib/python3.7/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


#### Getting evaluation metrics and evaluating model performance

In [49]:
confusion_matrix_test = confusion_matrix(y_test, test_pred)
confusion_matrix_train = confusion_matrix(y_train, train_pred)

print(confusion_matrix_train)
print(confusion_matrix_test)

[[33067     0]
 [ 7663     0]]
[[14149     1]
 [ 3306     0]]


#### Calculate Accuracy, True Positive Rate and True Negative Rates

In [50]:
Accuracy_Train=(confusion_matrix_train[0,0]+confusion_matrix_train[1,1])/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1]+confusion_matrix_train[1,0]+confusion_matrix_train[1,1])
TNR_Train= confusion_matrix_train[0,0]/(confusion_matrix_train[0,0]+confusion_matrix_train[0,1])
TPR_Train= confusion_matrix_train[1,1]/(confusion_matrix_train[1,0]+confusion_matrix_train[1,1])

print("Train TNR: ",TNR_Train)
print("Train TPR: ",TPR_Train)
print("Train Accuracy: ",Accuracy_Train)

Train TNR:  1.0
Train TPR:  0.0
Train Accuracy:  0.8118585808986005


In [51]:
Accuracy_Test=(confusion_matrix_test[0,0]+confusion_matrix_test[1,1])/(confusion_matrix_test[0,0]+confusion_matrix_test[0,1]+confusion_matrix_test[1,0]+confusion_matrix_test[1,1])
TNR_Test= confusion_matrix_test[0,0]/(confusion_matrix_test[0,0] +confusion_matrix_test[0,1])
TPR_Test= confusion_matrix_test[1,1]/(confusion_matrix_test[1,0] +confusion_matrix_test[1,1])

print("Test TNR: ",TNR_Test)
print("Test TPR: ",TPR_Test)
print("Test Accuracy: ",Accuracy_Test)

Test TNR:  0.9999293286219081
Test TPR:  0.0
Test Accuracy:  0.8105522456461961
